# Scrape Texas death row inmates, part 2

The table we scraped in the last notebook _probably_ could have been imported directly into Excel without too much trouble. But what if you also wanted to append a few columns of information from each inmate's detail page, as well?

In this section, we're going to supplement the scraper we just wrote with a _function_ that extracts data from inmates' detail pages. We're also going to use Python's built-in `time.sleep` function to pause for a few seconds between each row to give the government's servers a break.

First, let's import the libraries we'll need.

In [29]:
import requests
from bs4 import BeautifulSoup
import csv
import time

## Let's write a function

We need a function that will take a URL of a detail page and do these things:

- Open the detail page URL using `requests`
- Parse the contents using `BeautifulSoup`
- Isolate the bits of information we're interested in: height, weight, eye color, hair color, native county, native state, link to mugshot
- Return those bits of information to the script that called the function -- let's use a dictionary

We shall call our function `inmateDetails()`.

In [33]:
def inmateDetails(url):

    # create a dictionary with some default values
    out_dict = {
        'Height': None,
        'Weight': None,
        'Eye Color': None,
        'Hair Color': None,
        'Native County': None,
        'Native State': None,
        'mug': None
    }
    
    if not url.endswith('.html'):
        return out_dict
    
    # get the page
    r = requests.get(url)
    
    # soup the HTML
    soup = BeautifulSoup(r.text, 'html.parser')

    # find the table of info
    table = soup.find('table', {'class': 'tabledata_deathrow_table'})
    
    # target the mugshot, if it exists
    mug = table.find('img', {'class': 'photo_border_black_right'})
    
    # if there is a mug, grab the src and add it to the dict
    if mug:
        out_dict['mug'] = 'http://www.tdcj.state.tx.us/death_row/dr_info/' + mug['src']

    # get a list of the "label" cells
    # on some pages, they're identified by the class 'tabledata_bold_align_right_deathrow'
    # on others, they're identified by the class 'tabledata_bold_align_right_unit'
    # so we need to see which one we're working with
    label_cells = table.find_all('td', {'class': 'tabledata_bold_align_right_deathrow'})
    
    if not label_cells:
        label_cells = table.find_all('td', {'class': 'tabledata_bold_align_right_unit'})        
    
    # a list of the things we're interested in -- should match exactly the text of the cells
    attr_list = ['Height', 'Weight', 'Eye Color', 'Hair Color', 'Native County', 'Native State']

    # loop over the list of label cells
    for cell in label_cells:
        
        # check to see if the cell text is in our list of attributes
        if cell.text.strip() in attr_list:
            
            # if so, find the value -- go up to the tr and search for the other td --
            # and add that attribute to our dictionary
            out_dict[cell.text.strip()] = cell.parent.find('td', {'class': 'tabledata_align_left_deathrow'}).text.strip()

    # return the dictionary to the script
    return(out_dict)

OK, now we have our function. Let's drop it in the scraper we wrote for the last session.

First, let's get back to the part where where we have the rows of the table stored as a variable:

In [18]:
url = 'http://www.tdcj.state.tx.us/death_row/dr_offenders_on_dr.html'

r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

dr_table = soup.find('table', {'class': 'os'})

dr_rows = dr_table.find_all('tr')[1:]

Now we're going to loop over the rows of the table again as we write to a file -- let's call it 'tx-death-row-with-details.csv' -- but this time, we're _also_ going to call the function we just wrote, `inmateDetails`, to grab some details from the detail page.

The details will be returned as a dictionary, and we'll add these values to the list that we write out to file instead of just dropping in the link to the detail page.

_Furthermore_, because we're adding an HTTP request to every loop iteration, we're going to use `time.sleep` to pause for a few seconds at the end of each loop.

In [34]:
with open('tx-death-row-with-details.csv', 'w') as outfile:
    writer = csv.writer(outfile)
    headers = ['id', 'last', 'first', 'dob', 'sex', 'race',
               'admission_date', 'county', 'offense_date',
               'height', 'weight', 'eye_color', 'hair_color',
               'native_county', 'native_state']
    
    writer.writerow(headers)
    
    for row in dr_rows:
        cols = row.find_all('td')

        id_number = cols[0].text
        
        print(id_number)
        last_name = cols[2].text
        first_name = cols[3].text
        dob = cols[4].text
        sex = cols[5].text
        race = cols[6].text
        date_received = cols[7].text
        county = cols[8].text
        date_offense = cols[9].text

        detail_link = 'http://www.tdcj.state.tx.us/death_row/' + cols[1].a['href']
        details = inmateDetails(detail_link)
        
        height = details['Height']
        weight = details['Weight']
        eye_color = details['Eye Color']
        hair_color = details['Hair Color']
        native_county = details['Native County']
        native_state = details['Native State']
        
        writer.writerow([id_number, detail_link, last_name, first_name, dob, sex,
                         race, date_received, county, date_offense, height, weight,
                         eye_color, hair_color, native_county, native_state])

        time.sleep(2)

    print('Done!')

999604
999603
999602
999601
999600
999599
999598
999597
999596
999595
999594
999593
999592
999591
999590
999589
999588
999587
999586
999585
999584
999582
999581
999580
999579
999578
999577
999576
999575
999573
999572
999571
999570
999569
999568
999567
999566
999565
999564
999563
999562
999561
999560
999559
999558
999557
999556
999554
999553
999551
999550
999549
999548
999547
999545
999544
999543
999542
999541
999538
999537
999536
999535
999534
999531
999529
999527
999526
999524
999523
999522
999520
999519
999518
999517
999516
999515
999514
999513
999512
999509
999508
999507
999506
999505
999502
999501
999498
999497
999495
999494
999493
999492
999490
999489
999488
999484
999482
999480
999477
999476
999473
999472
999469
999465
999464
999462
999461
999460
999459
999458
999455
999453
999450
999447
999446
999443
999442
999441
999436
999433
999423
999420
999416
999412
999411
999410
999406
999402
999399
999398
999396
999393
999392
999391
999390
999388
999386
999383
999381
999379
999376
999373